#  Segmenting and Clustering Neighborhoods in Toronto
In this Notebook I will gather information about Toronto neighborhoods from different sources (Wikipedia, Foursquares, etc.). I will use different Python libraries to gather and clean the data. From this new dataset I will |train a model (K nearsest neighbors) to cluster the neighborhoods based on the venues in that area. Then we will plot these finding on maps and figures to illustrate their meaning. 

## Part 1 - Scrapping Wikipedia and creating dataframe

In this part we will gather data from the wikipedia website on Torronto. The postal code, neighborhood and borough will be stored in a dataframe. 

https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, 

In [7]:
# importing the libraries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests as req

print('libraries imported succesfully')

libraries imported succesfully


In [19]:
# making a get request from wikipedia and storing the HTML data in beautifullsoup object
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_doc = req.get(url).text
soup = BeautifulSoup(html_doc, 'html5lib')

In [204]:

#To turn al the cells of the table into a list to iterate through
table_cells = soup.table.find_all('td')

#create a empty pandas dataframe to store the information
df_postal = pd.DataFrame()

# iterate through the cells to extract the data
for cells in table_cells:
    PostalCode = cells.b.text
    Borough_Neighborhood = cells.span.text
    
    try: #not all PostalCodes have a borough or neighborhoods associated with them 
        Borough = Borough_Neighborhood.split("(")[0]
        try:
            Neighborhood = Borough_Neighborhood.split("(")[1].replace(")", " ").replace('/', ',') + ", " + Borough_Neighborhood.split("(")[2].replace(")", " ").replace('/', ',')
        except:
            Neighborhood = Borough_Neighborhood.split("(")[1].replace(")", " ").replace('/', ',')
    except:
        Borough = 'NA'
        Neighborhood = 'NA'
    
    df_temp = pd.DataFrame(data=[[PostalCode, Borough, Neighborhood]])
    df_postal = pd.concat([df_postal, df_temp])
    
# cleaning dataframe
df_postal.rename(columns = {0:'PostalCode', 1:'Borough', 2:'Neighborhood'}, inplace = True) #name columns
df_postal = df_postal[df_postal.Borough != 'NA'] #drop all rows without bourough information
df_postal.reset_index(drop = True, inplace=True)


## review of dataframe
The data frame has 103 PostalCodes which is the same number Wikipedia suggest there are in Torronto. We can distinguish 15 different Boruoughs in the dataframe, some of which seem to be Mail processing centres we might nog use later on the analyses.

In [206]:
# check dataframe
print('there are ' + str(len(df_postal['PostalCode'])) + ' distinct Postal Codes in our dataframe')

print('From ' + str(len(pd.unique(df_postal['Borough']))) + ' different Boroughs')
print("")
print(pd.unique(df_postal['Borough']))

df_postal.head(5)


there are 103 distinct Postal Codes in our dataframe
From 15 different Boroughs

['North York' 'Downtown Toronto' "Queen's Park" 'Etobicoke' 'Scarborough'
 'East York' 'York' 'East Toronto' 'West Toronto' 'East YorkEast Toronto'
 'Central Toronto' 'MississaugaCanada Post Gateway Processing Centre'
 'Downtown TorontoStn A PO Boxes25 The Esplanade' 'EtobicokeNorthwest'
 'East TorontoBusiness reply mail Processing Centre969 Eastern']


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
